In [28]:
import boto3
import boto3.dynamodb
import streamlit as st
import pandas as pd
from functools import reduce   # Only in Python 3, omit this in Python 2.x
from decimal import *
import plotly.express as px  # interactive charts
import plotly
from datetime import datetime
import plotly.graph_objs as go


In [34]:
#setup database access
ddb = boto3.resource('dynamodb')

table = ddb.Table('esp32c6_data')

#accessing table contents
response = table.scan(
    AttributesToGet=[
        'time',
        'data',
    ],
    # Limit=10,
)


In [35]:
response

{'Items': [{'time': Decimal('1713255080745'),
   'data': {'pump_activated': Decimal('0'), 'humidity': Decimal('68')}},
  {'time': Decimal('1713255707162'),
   'data': {'pump_activated': Decimal('0'), 'humidity': Decimal('68')}},
  {'time': Decimal('1713169088509'),
   'data': {'pump_activated': Decimal('1'), 'humidity': Decimal('89')}},
  {'time': Decimal('1713255682198'),
   'data': {'pump_activated': Decimal('0'), 'humidity': Decimal('67')}},
  {'time': Decimal('1713169144727'),
   'data': {'pump_activated': Decimal('1'), 'humidity': Decimal('89')}},
  {'time': Decimal('1713169131978'),
   'data': {'pump_activated': Decimal('1'), 'humidity': Decimal('89')}},
  {'time': Decimal('1713169306113'),
   'data': {'pump_activated': Decimal('1'), 'humidity': Decimal('89')}},
  {'time': Decimal('1713255863965'),
   'data': {'pump_activated': Decimal('0'), 'humidity': Decimal('70')}},
  {'time': Decimal('1713255870112'),
   'data': {'pump_activated': Decimal('0'), 'humidity': Decimal('70')}},
 

In [36]:
pump_activation = {}
humidity_levels = {}

for iter, result in enumerate(response['Items']):
    epoch = int(result['time']/1000)
    date_iter = datetime.fromtimestamp(epoch)
    print(date_iter)
    pump_activation[date_iter] = reduce(lambda rst, x: rst * 10 + x, result['data']['pump_activated'].as_tuple().digits)
    humidity_levels[date_iter] = reduce(lambda rst, x: rst * 10 + x, result['data']['humidity'].as_tuple().digits)


2024-04-16 16:11:20
2024-04-16 16:21:47
2024-04-15 16:18:08
2024-04-16 16:21:22
2024-04-15 16:19:04
2024-04-15 16:18:51
2024-04-15 16:21:46
2024-04-16 16:24:23
2024-04-16 16:24:30
2024-04-15 16:18:26
2024-04-16 16:07:19
2024-04-15 16:18:45
2024-04-16 16:25:11
2024-04-16 16:07:07
2024-04-15 16:22:05
2024-04-15 16:02:53
2024-04-15 16:19:11
2024-04-15 16:18:14
2024-04-15 16:22:11
2024-04-15 16:21:27
2024-04-15 16:18:20
2024-04-15 16:21:33
2024-04-15 16:26:30
2024-04-16 16:07:25
2024-04-16 00:16:30
2024-04-16 16:07:13
2024-04-16 16:21:40
2024-04-15 16:18:58
2024-04-16 16:25:24
2024-04-15 16:26:42
2024-04-15 16:21:39
2024-04-16 00:16:41
2024-04-15 16:21:21
2024-04-16 16:07:01
2024-04-15 16:21:59
2024-04-16 16:25:17
2024-04-16 16:24:42
2024-04-16 16:21:34
2024-04-16 16:21:28
2024-04-16 16:24:49
2024-04-15 16:18:32
2024-04-16 16:11:14
2024-04-15 16:02:47
2024-04-15 16:26:36
2024-04-16 16:24:36


In [37]:
pump_pd = pd.DataFrame(list(pump_activation.items()), columns=['date', 'pump activated'])
humidity_pd = pd.DataFrame(list(humidity_levels.items()), columns=['date', 'humidity'])


In [44]:
fig = px.scatter(
                data_frame=pump_pd, y="pump activated", x="date"
)
fig.show()

In [45]:
df = humidity_pd.sort_values(by='date', ascending=True,inplace=True)


In [47]:
fig = px.line(
                data_frame=df, y="humidity", x="date",range_y=[0,100]
)
fig.show()